In [1]:
%load_ext autoreload
%autoreload 2

import os
os.chdir('C:/Users/Iasonas/PycharmProjects/EntityMatching_Thesis/DataFusion/tutorials')

import fuse
fuse_env = fuse.Fuse(verbose=True)
fuse_session = fuse_env.get_session('test')

# MATCHER='DEDUPE'
MATCHER='AHC'

C:\Python38\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')
C:\Python38\lib\site-packages\sklearn\base.py:310: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.23.2 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
C:\Users\Iasonas\AppData\Roaming\Python\Python38\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
INFO:summarizer.preprocessing.cleaner:'pattern' package not found; tag filters are not available for English
C:\Python38\lib\site-packages\sklearn\base.py:310: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.23.2 when using versi

In [2]:
if MATCHER=='DEDUPE':
    from matching.dedupe import DedupeMatcher
    dataset = fuse_session.ingest('../../datasets/current/observed.json',
                                  'fusion_test', matching_strategy=DedupeMatcher)

if MATCHER=='AHC':
    from matching.ahc_matcher import AgglomerativeHierarchicalClustering
    dataset = fuse_session.ingest('../../datasets/current/observed.json',
                               'fusion_test', matching_strategy=AgglomerativeHierarchicalClustering)


In [3]:
fuse_session.train_matchers('../../datasets/current/matched.json')

In [ ]:
# x=dataset.entity_attribute_matchers['Education'].attribute_to_simrank_matcher['university'].cluster_simrank_buffer
# {k: round(v, 3) for k, v in sorted(x.items(), key=lambda item: item[1], reverse=True)}
# dataset.entity_attribute_matchers['Education'].attribute_to_simrank_matcher['university'].get_simrank_score_of_tokens('Bsc','undergraduate_degree')
# dataset.entity_attribute_matchers['Education'].attribute_to_simrank_matcher['degree'].get_simrank_score_of_tokens('Bsc','undergraduate_degree')
# dataset.entity_attribute_matchers['Education'].attribute_to_simrank_matcher['degree'].get_simrank_score_of_tokens('doctorate','doctoral_degree')
# dataset.entity_attribute_matchers['Education'].attribute_to_simrank_matcher['degree'].get_simrank_score_of_tokens('doctorate','Bsc')
# dataset.entity_attribute_matchers['Education'].attribute_to_simrank_matcher['degree'].get_simrank_score_of_tokens('Msc','Bsc')
# x=dataset.entity_attribute_matchers['Working Experience'].attribute_to_simrank_matcher['title'].cluster_simrank_buffer
# {k: round(v, 3) for k, v in sorted(x.items(), key=lambda item: item[1], reverse=True)}
# dataset.entity_attribute_matchers['Working Experience'].attribute_to_simrank_matcher['title'].get_simrank_score_of_tokens('manager','developer')
# dataset.entity_attribute_matchers['Working Experience'].attribute_to_simrank_matcher['title'].get_simrank_score_of_tokens('coder','developer')
# dataset.entity_attribute_matchers['Working Experience'].attribute_to_simrank_matcher['title'].get_simrank_score_of_tokens('analyst','developer')
# dataset.entity_attribute_matchers['Working Experience'].attribute_to_simrank_matcher['title'].get_simrank_score_of_tokens('analyst','manager')
# dataset.entity_attribute_matchers['Working Experience'].attribute_to_simrank_matcher['title'].get_simrank_score_of_tokens('doctor','manager')
# dataset.entity_attribute_matchers['Working Experience'].attribute_to_simrank_matcher['title'].get_simrank_score_of_tokens('clinician','doctor')

In [4]:
fuse_session.match_observations()

In [5]:
tr_clusters = fuse_session.find_true_clusters()

INFO:global:Running truth discovery for 10 iterations.
INFO:global:Iteration = 0
INFO:global:Source weights = {'Location': {'internsteps.com': 1.0, 'find.jobs': 1.0, 'pro-profiles.com': 1.0, 'pipl.com': 1.0}, 'Education': {'internsteps.com': 1.0, 'find.jobs': 1.0, 'pro-profiles.com': 1.0, 'pipl.com': 1.0}, 'Name': {'internsteps.com': 1.0, 'find.jobs': 1.0, 'pro-profiles.com': 1.0, 'pipl.com': 1.0}, 'Working Experience': {'internsteps.com': 1.0, 'find.jobs': 1.0, 'pro-profiles.com': 1.0, 'pipl.com': 1.0}}
INFO:global:Iteration = 1
INFO:global:Source weights = {'Location': {'internsteps.com': 1.7117167615545186, 'find.jobs': 1.2452157628599851, 'pro-profiles.com': 1.7117167615545186, 'pipl.com': 1.1143606456362487}, 'Education': {'internsteps.com': 1.817077277212345, 'find.jobs': 2.674148649426528, 'pro-profiles.com': 1.730390522851763, 'pipl.com': 2.867898902044107}, 'Name': {'internsteps.com': 1.4240346891027376, 'find.jobs': 0.9267620317414502, 'pro-profiles.com': 2.517696472610991, '

In [ ]:
true_facts = fuse_session.find_true_facts(persist=True, no_weights=False)
for f in true_facts:
    print(f.canonicalTuple)

INFO:global:Running fusion for 10 iterations.
INFO:global:Iteration = 0
INFO:global:Source weights = {'Location': {'internsteps.com': 1.0, 'find.jobs': 1.0, 'pro-profiles.com': 1.0, 'pipl.com': 1.0}, 'Education': {'internsteps.com': 1.0, 'find.jobs': 1.0, 'pro-profiles.com': 1.0, 'pipl.com': 1.0}, 'Name': {'internsteps.com': 1.0, 'find.jobs': 1.0, 'pro-profiles.com': 1.0, 'pipl.com': 1.0}, 'Working Experience': {'internsteps.com': 1.0, 'find.jobs': 1.0, 'pro-profiles.com': 1.0, 'pipl.com': 1.0}}


In [ ]:
eval = fuse_session.evaluate(inferred_facts=true_facts,grd_path='../../datasets/current/eval.json')
print(eval)